# Computation of the dimension of the trace of associated generated Lie algebra

In [15]:
from sage.parallel.decorate import parallel

In [48]:
from tqdm import tqdm

In [17]:
@parallel
def lie_bracket(Vec1, Vec2, D):
    grad1 = [np.array(Vec1[k].gradient()) for k in range(D)]
    grad2 = [np.array(Vec2[k].gradient()) for k in range(D)]

    vector_vec1 = np.array(Vec1)
    vector_vec2 = np.array(Vec2)
    grad1 = np.array(grad1)
    grad2 = np.array(grad2)
    res = grad1 @ vector_vec2 - grad2 @ vector_vec1
    return res


@parallel
def evaluate_polynomial(vec, dic):
    # not used
    return np.array([vec[k](**dic) for k in range(len(vec))]).reshape(1, -1)


In [ ]:
def compute_vector_field(Vector_field, index, dic):
    res = Vector_field[index]
    evaluation = res(**dic)
    return res, evaluation  

@parallel
def compute_vector_field_list(Vector_field, dic, D):
    res =  [compute_vector_field(Vector_field, j, dic) for j in range(D)]
    return res

In [1]:
import time
import warnings

def dim_trace_lie_algebra(Vector_fields, verbose=0, n_known_functions=0, fast=True):
    if fast:
        Vector_fields_original = Vector_fields.copy() # can be useful to recompute the result without fast=True
        if not n_known_functions:
            warnings.warn("Running with fast=True, this may find too many converved functions. If the result matches n_known_functions, you can trust it. Otherwise, try again with fast=False.")

    if verbose > 3:
        print("Vector_fields", Vector_fields)
        print("Computing dim trace lie algebra")
    D, d = np.shape(Vector_fields)

    # construction of a list that contains all vector fields
    start_time = time.time()

    value = np.random.rand(D)
    dic = {}
    for k in range(D):
        dic[f"x{k+1}"] = value[k]

    Vector_fields = Vector_fields.T
    Vec = []
    evaluation = []

    if verbose:
        print(f"d: {d}")
    start_time = time.time()
    vector_field_list = list(compute_vector_field_list([(Vector_fields[i], dic, D) for i in range(d)]))
    if verbose:
        print(f"computed vector field list in {time.time() - start_time} seconds")
    for i in range(d):
        vector_field = vector_field_list[i][1] #[1] because of parallel
        vi, vi_valued = zip(*[elem for elem in vector_field])
        vi = vector(vi)
        Vec.append(vi) 
        evaluation.append(vi_valued)
        
    evaluation = np.array(evaluation)
    
    rank = np.linalg.matrix_rank(evaluation)
    
    if verbose:
        print("dimension d: ", d)
        print("dimension D :", D)
        print("parallel vector field computation done!")
        print(f"time to define vector fields: {time.time() - start_time}")
     
    # computation of the dimension of the trace of the Lie algebra
        
    # construction of V_{i+1} = V_i + [Vec, V_i]
    # and evaluation of the dimension of its trace, we stop as soon as it stagnates
    Vec = np.array(Vec) 
    Vec_pivot = Vec
    new_Vec = Vec
    u = rank - 1
    step = 0
    vec_eval = None
    if verbose:
        print("Vec", Vec.shape)
        print("rank", rank)
        print("D", D)
        print("n_known_functions", n_known_functions)
    vec_eval = evaluation
    while rank > u and rank < D - n_known_functions:
        if verbose:
            print("step of construction", step+1)
            print(rank)
            print("upper bound", D - rank)
        u = rank
        start_time = time.time()
        A = []
        if verbose > 1:
            if not fast:
                print(f"launching {len(new_Vec) * len(Vec_pivot)} lie brackets computations")
            else:
                print(f"launching {len(new_Vec) * min(5, len(Vec_pivot))} lie brackets computations")
        lie_time = time.time()
        if not fast:
            vec_res_list = list(lie_bracket([(new_Vec[i], Vec_pivot[j], D) for i in range(len(new_Vec)) for j in range(len(Vec_pivot))]))
        else:
            #empirically, this doesn't change the result but is much faster
            # if the result is weird, try again without this
            vec_res_list = list(lie_bracket([(new_Vec[i], Vec_pivot[j], D) for i in range(len(new_Vec)) for j in range(min(3, len(Vec_pivot)))]))
        if verbose > 1:
            print(f"lie bracket time: {time.time() - lie_time}")
        time_eval = time.time()
        for vec_res_ in vec_res_list:
            vec_res = vec_res_[1]
            vec_res_valued = np.array([vec_res[k](**dic) for k in range(len(vec_res))]).reshape(1, -1)
            if vec_eval is None:
                new_vec_eval = vec_res_valued
                rank_vec_eval = 0
            else:
                new_vec_eval = np.concatenate((vec_eval, vec_res_valued), 0)
                rank_vec_eval = np.linalg.matrix_rank(vec_eval)
            new_rank = np.linalg.matrix_rank(new_vec_eval)
            if new_rank > rank_vec_eval:
                A.append(vec_res)
                vec_eval = new_vec_eval
                if not n_known_functions is None and new_rank >= D - n_known_functions:
                    rank = new_rank
                    if verbose:
                        print("n_known_functions reached on step", step)
                    return rank
            else:
                pass
        if verbose > 1:
            print(f"evaluation rank time: {time.time() - time_eval}")
                            
        if len(A) == 0:
            if verbose:
                print(f"no new vector field on step {step}")
            break

        new_Vec = np.array(A)
        rank = new_rank
        
        if verbose:
            print("rank end", rank)
            print("rank before", u)
            print("new vec", new_Vec.shape)
        step +=1
    if rank < D - n_known_functions and fast:
        # try again without fast
        return dim_trace_lie_algebra(Vector_fields_original, verbose=verbose, n_known_functions=n_known_functions, fast=False)
    return rank